In [14]:
import numpy as np
import L_S_decomp as ls

A, B, C = ls.l_s(10, 10, 2, 2)

In [9]:
# This code is from https://github.com/nwbirnie/rpca/blob/master/rpca.py

import math
import numpy.linalg

def robust_pca(M):
    """ 
    Decompose a matrix into low rank and sparse components.
    Computes the RPCA decomposition using Alternating Lagrangian Multipliers.
    Returns L,S the low rank and sparse components respectively
    """
    L = numpy.zeros(M.shape)
    S = numpy.zeros(M.shape)
    Y = numpy.zeros(M.shape)
    print (M.shape)
    mu = (M.shape[0] * M.shape[1]) / (4.0 * L1Norm(M))
    lamb = max(M.shape) ** -0.5
    while not converged(M,L,S):
        L = svd_shrink(M - S - (mu**-1) * Y, mu)
        S = shrink(M - L + (mu**-1) * Y, lamb * mu)
        Y = Y + mu * (M - L - S)
    return L,S
    
def svd_shrink(X, tau):
    """
    Apply the shrinkage operator to the singular values obtained from the SVD of X.
    The parameter tau is used as the scaling parameter to the shrink function.
    Returns the matrix obtained by computing U * shrink(s) * V where 
        U are the left singular vectors of X
        V are the right singular vectors of X
        s are the singular values as a diagonal matrix
    """
    U,s,V = numpy.linalg.svd(X, full_matrices=False)
    return numpy.dot(U, numpy.dot(numpy.diag(shrink(s, tau)), V))
    
def shrink(X, tau):
    """
    Apply the shrinkage operator the the elements of X.
    Returns V such that V[i,j] = max(abs(X[i,j]) - tau,0).
    """
    V = numpy.copy(X).reshape(X.size)
    for i in range(V.size):
        V[i] = math.copysign(max(abs(V[i]) - tau, 0), V[i])
        if V[i] == -0:
            V[i] = 0
    return V.reshape(X.shape)
            
def frobeniusNorm(X):
    """
    Evaluate the Frobenius norm of X
    Returns sqrt(sum_i sum_j X[i,j] ^ 2)
    """
    accum = 0
    V = numpy.reshape(X,X.size)
    for i in range(V.size):
        accum += abs(V[i] ** 2)
    return math.sqrt(accum)

def L1Norm(X):
    """
    Evaluate the L1 norm of X
    Returns the max over the sum of each column of X
    """
    return max(numpy.sum(X,axis=0))

def converged(M,L,S):
    """
    A simple test of convergence based on accuracy of matrix reconstruction
    from sparse and low rank parts
    """
    error = frobeniusNorm(M - L - S) / frobeniusNorm(M)
    print ("error =", error)
    return error <= 10e-6

In [10]:
A


array([[ 0.0116945 , -0.70532166,  0.13698004, -0.6228767 , -0.61049191,
        -1.79402145,  0.4049477 , -3.71628847,  5.18205452, -2.97593052],
       [ 0.81586119, -0.07263369,  0.49922701, -2.17344863, -4.14421274,
         1.66895207, -1.49124059, -4.49685851, -0.71767303, -7.69316756],
       [-2.04697441, -2.04765537, -5.95523988,  1.63384302, -1.12103277,
        -0.155596  , -0.98628735, -0.11881662, -0.67893617,  0.23321236],
       [-1.05200613,  0.87277235,  6.12564493, -2.5604271 , -4.67447433,
         1.81200271,  2.91010702, -1.85421898,  1.49102638, -0.37937549],
       [ 0.96432929,  1.94839836,  3.87978593, -1.31498477,  1.89078083,
         1.75980448,  1.25851233,  0.62620782, -4.58481326, -4.40068192],
       [ 0.03063996,  1.90426074,  2.24427604,  1.0781321 ,  1.61421564,
        -3.52272183, -2.30998012, -1.65398914,  2.16692434, -0.56270688],
       [ 1.9653036 ,  1.93161227, -3.34714838, -0.56313788,  0.80696086,
         1.31677969,  0.07437734,  4.501739  

In [12]:
L0, S0 = robust_pca(A)

(10, 10)
error = 1.0
error = 0.22176894452499224
error = 0.11828588515777007
error = 0.08202087069651134
error = 0.023047247915602583
error = 0.03023930329571088
error = 0.015437157625456808
error = 0.008114915149764219
error = 0.008395149117226126
error = 0.011270412114425251
error = 0.016901733726425748
error = 0.0016668127308908005
error = 1.1160087606299598e-16


In [15]:
np.linalg.norm(B - L0, ord="fro")/np.linalg.norm(L0, ord="fro")

2.084273163013068

In [16]:
np.linalg.norm(C - S0, ord="fro")/np.linalg.norm(S0, ord="fro")

0.9949101801885609

In [18]:
np.linalg.norm(A - L0 - S0, ord="fro")/np.linalg.norm(A, ord="fro")

1.3314658959673502

In [19]:
A


array([[ 1.79336167,  0.13422899,  3.39010196,  1.07946677, -3.07044227,
         1.47410101, -0.74268769, -0.66749555, -2.63215866,  0.4409927 ],
       [-0.64173311, -3.2892044 ,  2.22457862,  1.15359425, -1.23808115,
        -0.60939009, -4.72357269,  2.89933529, -0.49128408,  3.32152363],
       [ 1.74991267, -0.91175276, -1.09220847,  1.89493743, -1.23815678,
        -0.25151487, -0.4602313 ,  1.14235494,  1.27220518,  0.37429999],
       [-2.42633892, -2.51137769, -1.80429128,  4.5926138 ,  4.47472278,
         3.37800903, -0.90233346,  1.16570419,  3.72047867,  6.74433343],
       [-4.03109365, -2.22009291,  3.23729184,  0.86230448,  0.49440207,
         3.75638552, -7.09442151,  0.76802001, -4.01813525,  1.46236219],
       [-3.02932323, -0.90812648,  0.678653  ,  0.43518593, -2.23135771,
        -2.14241094, -0.67308034,  2.52629545, -1.9003267 , -0.32039029],
       [ 2.2771758 , -5.65624476,  3.06342672,  1.08425248, -1.94733802,
        -1.28528913, -5.64286172,  2.72351538

In [20]:
L0+S0


array([[ 0.0116945 , -0.70532166,  0.13698004, -0.6228767 , -0.61049191,
        -1.79402145,  0.4049477 , -3.71628847,  5.18205452, -2.97593052],
       [ 0.81586119, -0.07263369,  0.49922701, -2.17344863, -4.14421274,
         1.66895207, -1.49124059, -4.49685851, -0.71767303, -7.69316756],
       [-2.04697441, -2.04765537, -5.95523988,  1.63384302, -1.12103277,
        -0.155596  , -0.98628735, -0.11881662, -0.67893617,  0.23321236],
       [-1.05200613,  0.87277235,  6.12564493, -2.5604271 , -4.67447433,
         1.81200271,  2.91010702, -1.85421898,  1.49102638, -0.37937549],
       [ 0.96432929,  1.94839836,  3.87978593, -1.31498477,  1.89078083,
         1.75980448,  1.25851233,  0.62620782, -4.58481326, -4.40068192],
       [ 0.03063996,  1.90426074,  2.24427604,  1.0781321 ,  1.61421564,
        -3.52272183, -2.30998012, -1.65398914,  2.16692434, -0.56270688],
       [ 1.9653036 ,  1.93161227, -3.34714838, -0.56313788,  0.80696086,
         1.31677969,  0.07437734,  4.501739  